In [13]:
# prep: import modules and get pwd
import pymysql
from sqlalchemy import create_engine
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
import getpass  # To get the password without showing the input
password = getpass.getpass()

········


In [19]:
# get the data
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)
query = '''SELECT f.film_id, f.title, f.release_year, f.length, f.rating, f.rental_rate,
       c.name AS category, a.actor_id, a.first_name, a.last_name,
       COUNT(r.rental_id) AS rental_count
FROM film f
JOIN film_category fc ON f.film_id = fc.film_id
JOIN category c ON fc.category_id = c.category_id
JOIN film_actor fa ON f.film_id = fa.film_id
JOIN actor a ON fa.actor_id = a.actor_id
JOIN inventory i ON f.film_id = i.film_id
JOIN rental r ON i.inventory_id = r.inventory_id
GROUP BY f.film_id, f.title, f.release_year, f.length, f.rating,
         c.name, a.actor_id, a.first_name, a.last_name;
'''

df = pd.read_sql_query(query, engine)
df.head()

,film_id,title,release_year,length,rating,rental_rate,category,actor_id,first_name,last_name,rental_count
0,1,ACADEMY DINOSAUR,2006,86,PG,0.99,Documentary,1,PENELOPE,GUINESS,23
1,1,ACADEMY DINOSAUR,2006,86,PG,0.99,Documentary,10,CHRISTIAN,GABLE,23
2,1,ACADEMY DINOSAUR,2006,86,PG,0.99,Documentary,20,LUCILLE,TRACY,23
3,1,ACADEMY DINOSAUR,2006,86,PG,0.99,Documentary,30,SANDRA,PECK,23
4,1,ACADEMY DINOSAUR,2006,86,PG,0.99,Documentary,40,JOHNNY,CAGE,23


In [20]:
query_unique_films = '''SELECT f.title, IF(r.rental_date IS NOT NULL, TRUE, FALSE) AS rented_in_may
        FROM film f
        LEFT JOIN inventory i ON f.film_id = i.film_id
        LEFT JOIN rental r ON i.inventory_id = r.inventory_id
        AND YEAR(r.rental_date) = 2005 AND MONTH(r.rental_date) = 5;
        ;
        '''

data1 = pd.read_sql_query(query_unique_films, engine)
data1.head()

,title,rented_in_may
0,ACADEMY DINOSAUR,0
1,ACADEMY DINOSAUR,1
2,ACADEMY DINOSAUR,0
3,ACADEMY DINOSAUR,0
4,ACADEMY DINOSAUR,0


In [21]:
# Encode categorical variables
categorical_cols = ['category', 'first_name', 'last_name']
label_encoders = {}
for col in categorical_cols:
    label_encoders[col] = LabelEncoder()
    df[col] = label_encoders[col].fit_transform(df[col])

# Scale numerical variables
numerical_cols = ['release_year', 'length', 'rating', 'rental_rate', 'rental_count']
scaler = StandardScaler()
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

print(df.head())

ValueError: could not convert string to float: 'PG'

In [ ]:
# Split the data
X = df.drop('rented_in_may', axis=1)
y = df['rented_in_may']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#logistic regression model
logreg = LogisticRegression()

# Train the model
logreg.fit(X_train, y_train)

# Make predictions
y_pred = logreg.predict(X_test)

# Evaluate the model's accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)